# Predicting sentiment from product reviews


The goal of this notebook is to explore logistic regression.

In this notebook you will use product review data from Amazon.com to predict whether the sentiments about a product (from its reviews) are positive or negative.

* Use Pandas to do some feature engineering
* Train a logistic regression model to predict the sentiment of product reviews.
* Inspect the weights (coefficients) of a trained logistic regression model.
* Make a prediction (both class and probability) of sentiment for products review.
* Given the logistic regression weights, predictors and ground truth labels, compute the **accuracy** of the model.
* Inspect the coefficients of the logistic regression model and interpret their meanings.

Let's get started!
    
## Fire up Pandas and Scikit

In [1]:
from sklearn import linear_model as lm
from sklearn.feature_extraction import text
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer # It will be used to create a observation_word_count sparseMatrix
import numpy as np
import pandas as pd
import math
import string
import re # Regular Expressions library
from collections import Counter # High-performance container datatypes
np.random.seed(1234)


# Data preparation

We will use a dataset consisting of baby product reviews on Amazon.com.

In [2]:
dtype_dict = {'name':str, 'review':str, 'rating':int}
products = pd.read_csv('amazon_baby.csv',dtype=dtype_dict)
products.head()

,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5


Now, let us see a preview of what the dataset looks like.

In [3]:
products.dtypes

name      object
review    object
rating     int32
dtype: object

Let us explore a specific example of a baby product.


In [4]:
products.loc[260]

name               The First Years Massaging Action Teether
review    I've bought several teethers and this i one of...
rating                                                    5
Name: 260, dtype: object

## Build the word count vector for each review

Now, we will perform 2 simple data transformations:

1. Remove punctuation using [Python's built-in](https://docs.python.org/2/library/string.html) string functionality.
2. Transform the reviews into word-counts.

**Aside**. In this notebook, we remove all punctuations for the sake of simplicity. A smarter approach to punctuations would preserve phrases such as "I'd", "would've", "hadn't" and so forth. 

In [5]:
def count_words(s):
    # Pass all the words to lowercase and split them
    word_list = str(s).lower().split()
    #Use regular expressions to take only the alphanumeric words values
    words = re.findall(r'\w+', str(word_list))
    #Use Counter to get the word count
    return Counter(words)
def strip_punctuation(s):
    return ''.join([i for i in str(s) if i not in string.punctuation])

Apply the functions to create 2 new columns

In [6]:
products['review_clean'] = products['review'].apply(strip_punctuation)
products['word_count'] = products['review_clean'].apply(count_words)

In [7]:
products.head()

,name,review,rating,review_clean,word_count
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3,These flannel wipes are OK but in my opinion n...,"{'these': 1, 'flannel': 1, 'wipes': 3, 'are': ..."
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5,it came early and was not disappointed i love ...,"{'it': 3, 'came': 1, 'early': 1, 'and': 3, 'wa..."
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5,Very soft and comfortable and warmer than it l...,"{'very': 1, 'soft': 1, 'and': 2, 'comfortable'..."
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5,This is a product well worth the purchase I h...,"{'this': 4, 'is': 4, 'a': 2, 'product': 2, 'we..."
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,All of my kids have cried nonstop when I tried...,"{'all': 2, 'of': 1, 'my': 1, 'kids': 2, 'have'..."


Use '' to fill the NaN reviews in case is any

In [8]:
products = products.fillna({'review':''})

Now, let us explore what the sample example above looks like after these 2 transformations. Here, each entry in the **word_count** column is a dictionary where the key is the word and the value is a count of the number of times the word occurs.

In [9]:
print(products.loc[11]['review_clean'],"\n",products.loc[11]['word_count'])

This book is perfect  Im a first time new mom and this book made it so easy to keep track of feedings diaper changes sleep  Definitely would recommend this for new moms  Plus its small enough that I throw in the diaper back for doctor visits 
 Counter({'this': 3, 'book': 2, 'new': 2, 'diaper': 2, 'for': 2, 'is': 1, 'perfect': 1, 'im': 1, 'a': 1, 'first': 1, 'time': 1, 'mom': 1, 'and': 1, 'made': 1, 'it': 1, 'so': 1, 'easy': 1, 'to': 1, 'keep': 1, 'track': 1, 'of': 1, 'feedings': 1, 'changes': 1, 'sleep': 1, 'definitely': 1, 'would': 1, 'recommend': 1, 'moms': 1, 'plus': 1, 'its': 1, 'small': 1, 'enough': 1, 'that': 1, 'i': 1, 'throw': 1, 'in': 1, 'the': 1, 'back': 1, 'doctor': 1, 'visits': 1})


## Extract sentiments

We will **ignore** all reviews with *rating = 3*, since they tend to have a neutral sentiment.

In [10]:
products = products[products['rating'] != 3]
len(products)

166752

Now, we will assign reviews with a rating of 4 or higher to be *positive* reviews, while the ones with rating of 2 or lower are *negative*. For the sentiment column, we use +1 for the positive class label and -1 for the negative class label.

In [11]:
products['sentiment'] = products['rating'].apply(lambda rating : +1 if rating > 3 else -1)
products.sample(n=5)

,name,review,rating,review_clean,word_count,sentiment
136001,*The Art of CureTM *SAFETY KNOTTED* - Mixed Co...,I am not totally certain about it's effects in...,5,I am not totally certain about its effects in ...,"{'i': 3, 'am': 1, 'not': 1, 'totally': 1, 'cer...",1
70358,American Baby Company Organic Cotton Quilted M...,"We initially just bought one, used it so much ...",5,We initially just bought one used it so much b...,"{'we': 4, 'initially': 1, 'just': 2, 'bought':...",1
113748,T Rex Night Light,Kind of cheesy looking... and very 1980ies. An...,2,Kind of cheesy looking and very 1980ies An old...,"{'kind': 1, 'of': 1, 'cheesy': 1, 'looking': 1...",-1
35141,Pretty Pique Round Crib Bedding - Color Pink,I love this Pique crib bedding. It's a stunnin...,5,I love this Pique crib bedding Its a stunning ...,"{'i': 1, 'love': 1, 'this': 1, 'pique': 1, 'cr...",1
68005,Boppy Rock in Comfort Travel Swing - Pink,This is the only swing we need for our Little ...,5,This is the only swing we need for our Little ...,"{'this': 7, 'is': 8, 'the': 16, 'only': 3, 'sw...",1


Now, we can see that the dataset contains an extra column called **sentiment** which is either positive (+1) or negative (-1).

## Split data into training and test sets

There are 2 extra files. The train_idx and test_idx on json files. Use them to obtain the training and testing observations

- The two .json file containing splited data indexes in the products DF

In [12]:
trainIDX = pd.read_json('classification-train-idx.json')
trainIDX.sample(n=5)

,0
76307,95250
2106,2646
124875,156044
70593,88080
34715,43347


In [13]:
testIDX = pd.read_json('classification-test-idx.json')
testIDX.sample(n=5)

,0
10195,51152
23909,119672
8454,42488
29769,148932
26986,135113


Since the values are the indexes to keep, use iloc to obtain only the observations specified for each dataframe (train and test)

In [14]:
X_train = products.iloc[trainIDX[0].values]
X_train.head()

,name,review,rating,review_clean,word_count,sentiment
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5,it came early and was not disappointed i love ...,"{'it': 3, 'came': 1, 'early': 1, 'and': 3, 'wa...",1
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5,Very soft and comfortable and warmer than it l...,"{'very': 1, 'soft': 1, 'and': 2, 'comfortable'...",1
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5,This is a product well worth the purchase I h...,"{'this': 4, 'is': 4, 'a': 2, 'product': 2, 'we...",1
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,All of my kids have cried nonstop when I tried...,"{'all': 2, 'of': 1, 'my': 1, 'kids': 2, 'have'...",1
5,Stop Pacifier Sucking without tears with Thumb...,"When the Binky Fairy came to our house, we did...",5,When the Binky Fairy came to our house we didn...,"{'when': 2, 'the': 6, 'binky': 3, 'fairy': 3, ...",1


In [15]:
X_test = products.iloc[testIDX[0].values]
X_test.head()

,name,review,rating,review_clean,word_count,sentiment
9,"Baby Tracker&reg; - Daily Childcare Journal, S...",This has been an easy way for my nanny to reco...,4,This has been an easy way for my nanny to reco...,"{'this': 3, 'has': 1, 'been': 2, 'an': 1, 'eas...",1
10,"Baby Tracker&reg; - Daily Childcare Journal, S...",I love this journal and our nanny uses it ever...,4,I love this journal and our nanny uses it ever...,"{'i': 7, 'love': 2, 'this': 2, 'journal': 1, '...",1
16,Nature's Lullabies First Year Sticker Calendar,"I love this little calender, you can keep trac...",5,I love this little calender you can keep track...,"{'i': 2, 'love': 1, 'this': 1, 'little': 1, 'c...",1
20,Nature's Lullabies Second Year Sticker Calendar,I had a hard time finding a second year calend...,5,I had a hard time finding a second year calend...,"{'i': 5, 'had': 1, 'a': 6, 'hard': 1, 'time': ...",1
28,"Lamaze Peekaboo, I Love You","One of baby's first and favorite books, and it...",4,One of babys first and favorite books and it i...,"{'one': 1, 'of': 1, 'babys': 1, 'first': 1, 'a...",1


In [16]:
print(len(X_train),len(X_test))

133416 33336


As we can see, is about 20% of the observations

## Now, let's split the features from the target
The target will be the column 'sentiment', to check if it is a good or bad feeling on the review

In [17]:
X_trainy = X_train['sentiment']
X_train = X_train[X_train.columns.drop('sentiment')]

In [18]:
X_train.sample()

,name,review,rating,review_clean,word_count
114026,Motorola Digital Audio Baby Monitor,We have a rather large two story house (3500 s...,5,We have a rather large two story house 3500 sq...,"{'we': 5, 'have': 1, 'a': 3, 'rather': 1, 'lar..."


In [19]:
X_testy = X_test['sentiment']
X_test = X_test[X_test.columns.drop('sentiment')]

# Train a sentiment classifier with logistic regression

We will now use logistic regression to create a sentiment classifier on the training data. This model will use the column **word_count** as a feature and the column **sentiment** as the target.
**Note:** This block may take 1-2 minutes.

In [20]:
# Use this token pattern to keep single-letter words
vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
    
    # fit_transform, learns vocabulary from the training data and assign columns to words
    # Then convert the training data into a sparse matrix
train_matrix = vectorizer.fit_transform(X_train['review_clean'])

    # Second, convert the test data into a sparse matrix, using the same word-column mapping.
    # if any word on the test is not present on the trained sparse_matrix, it will not
    # count it
test_matrix = vectorizer.transform(X_test['review_clean'])

# Fit a model!
Finally, after this process, we can make a logistic regression using the sparse matrix previously obtained.
In this case, is important to take in consideration that instead of calling the features, we will use
the sparse matrix as features **Note:** It could take a couple of minutes too

In [21]:
sentiment_model = lm.LogisticRegression()
sentiment_model.fit(train_matrix,X_trainy)

D:\Anaconda2\Anaconda\envs\mlcc\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\Anaconda2\Anaconda\envs\mlcc\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [22]:
sentiment_model.coef_.shape

(1, 121713)

There are a total of `121713` coefficients in the model. Recall from the lecture that positive weights $w_j$ correspond to weights that cause positive sentiment, while negative weights correspond to negative sentiment. 

Fill in the following blocks of code to calculate how many *weights* are positive ( >= 0). (**Hint**: The `'value'` column in DataFrame *weights* must be positive ( >= 0)).

In [23]:
poscount = 0
for i in range(121713):
    if sentiment_model.coef_[0][i] > 0:
        poscount += 1
print(poscount)

85802


Now that we have fitted the model, we can extract the weights (coefficients) 

In [24]:
weights = sentiment_model.coef_.ravel()

In [25]:
weights = pd.DataFrame({'value':weights})
weights.head()

,value
0,-1.236477
1,0.000206
2,0.025447
3,0.006200
4,0.000046


In [26]:
num_positive_weights = sum(weights['value'].apply(lambda value : 1 if value > 0 else 0))
num_negative_weights = sum(weights['value'].apply(lambda value : 1 if value < 0 else 0))

print("Number of positive weights: %s " % num_positive_weights)
print("Number of negative weights: %s " % num_negative_weights)

Number of positive weights: 85802 
Number of negative weights: 35911 


**Quiz question:** How many weights are >= 0?
* 87053

## Making predictions with logistic regression

Now that a model is trained, we can make predictions on the **test data**. In this section, we will explore this in the context of 3 examples in the test dataset.  We refer to this set of 3 examples as the **sample_test_data**.

In [27]:
sample_test_data = X_test[10:13]
print(sample_test_data)

                                                 name  \
59                          Our Baby Girl Memory Book   
71  Wall Decor Removable Decal Sticker - Colorful ...   
91  New Style Trailing Cherry Blossom Tree Decal R...   

                                               review  rating  \
59  Absolutely love it and all of the Scripture in...       5   
71  Would not purchase again or recommend. The dec...       2   
91  Was so excited to get this product for my baby...       1   

                                         review_clean  \
59  Absolutely love it and all of the Scripture in...   
71  Would not purchase again or recommend The deca...   
91  Was so excited to get this product for my baby...   

                                           word_count  
59  {'absolutely': 1, 'love': 1, 'it': 2, 'and': 2...  
71  {'would': 2, 'not': 2, 'purchase': 1, 'again':...  
91  {'was': 2, 'so': 1, 'excited': 1, 'to': 3, 'ge...  


Let's dig deeper into the first row of the **sample_test_data**. Here's the full review:

In [28]:
sample_test_data.iloc[0]['review']

'Absolutely love it and all of the Scripture in it.  I purchased the Baby Boy version for my grandson when he was born and my daughter-in-law was thrilled to receive the same book again.'

That review seems pretty positive.

Now, let's see what the next row of the **sample_test_data** looks like. As we could guess from the rating (2), the review is quite negative.

In [29]:
sample_test_data.iloc[1]['review']

'Would not purchase again or recommend. The decals were thick almost plastic like and were coming off the wall as I was applying them! The would NOT stick! Literally stayed stuck for about 5 minutes then started peeling off.'

In [30]:
sample_test_data.iloc[2]['review']

"Was so excited to get this product for my baby girls bedroom!  When I got it the back is NOT STICKY at all!  Every time I walked into the bedroom I was picking up pieces off of the floor!  Very very frustrating!  Ended up having to super glue it to the wall...very disappointing.  I wouldn't waste the time or money on it."

We will now make a **class** prediction for the **sample_test_data**. The `sentiment_model` should predict **+1** if the sentiment is positive and **-1** if the sentiment is negative. Recall from the lecture that the **score** (sometimes called **margin**) for the logistic regression model  is defined as:

$$
\mbox{score}_i = \mathbf{w}^T h(\mathbf{x}_i)
$$ 

where $h(\mathbf{x}_i)$ represents the features for example $i$.  We will use some functions to obtain the **scores** using Scikit-learn. For each row, the **score** (or margin) is a number in the range **[-inf, inf]**. <p>
The <b>decision_function</b> tells us on which side of the hyperplane generated by the classifier we are (and how far we are away from it)

In [31]:
sample_test_matrix = vectorizer.transform(sample_test_data['review_clean'])
scores = sentiment_model.decision_function(sample_test_matrix)
scores = pd.DataFrame(scores)
scores

,0
0,5.566628
1,-3.171039
2,-10.429979


### Predicting sentiment

These scores can be used to make class predictions as follows:

$$
\hat{y} = 
\left\{
\begin{array}{ll}
      +1 & \mathbf{w}^T h(\mathbf{x}_i) > 0 \\
      -1 & \mathbf{w}^T h(\mathbf{x}_i) \leq 0 \\
\end{array} 
\right.
$$

Using scores, write code to calculate $\hat{y}$, the class predictions:

In [32]:
y_hat = scores[0].apply(lambda value : 1 if value > 0 else -1)
y_hat

0    1
1   -1
2   -1
Name: 0, dtype: int64

Or use the predict function of the model as it is meant to be

In [33]:
sentiment_model.predict(sample_test_matrix)

array([ 1, -1, -1], dtype=int64)

### Probability predictions

Recall from the lectures that we can also calculate the probability predictions from the scores using:
$$
P(y_i = +1 | \mathbf{x}_i,\mathbf{w}) = \frac{1}{1 + \exp(-\mathbf{w}^T h(\mathbf{x}_i))}.
$$

Using the variable **scores** calculated previously, write code to calculate the probability that a sentiment is positive using the above formula. For each row, the probabilities should be a number in the range **[0, 1]**.

In [34]:
Probs = 1-(1/(1+np.exp(scores)))
print(Probs)

          0
0  0.996191
1  0.040270
2  0.000030


**Checkpoint**: Make sure your probability predictions match the ones obtained from the model functions.

In [35]:
print(sentiment_model.predict_proba(sample_test_matrix))

[[3.80878928e-03 9.96191211e-01]
 [9.59729773e-01 4.02702268e-02]
 [9.99970467e-01 2.95328277e-05]]


** Question:** Of the three data points in **sample_test_data**, which one (first, second, or third) has the **lowest probability** of being classified as a positive review?
* The third one

# Find the most positive (and negative) review

We now turn to examining the full test dataset, **test_data**, and use Scikit-Learn to form predictions on all of the test data points for faster performance.

Using the `sentiment_model`, find the 10 reviews in the entire **test_data** with the **highest probability** of being classified as a **positive review**. We refer to these as the "most positive reviews."

To calculate these top-20 reviews, use the following steps:
1.  Make probability predictions on **test_data** using the `sentiment_model`. (**Hint:** Instead of calling `.predict` to make predictions on the test data, call `.predict.proba` to output the probability rather than just the most likely class.)
2.  Sort the data according to those predictions and pick the top 10.

In [36]:
X_test['prob'] = sentiment_model.predict_proba(test_matrix)[:,1]
X_test.sort_values('prob',ascending=False,inplace=True)
X_test.head(10)

,name,review,rating,review_clean,word_count,prob
147949,"Baby Jogger City Mini GT Single Stroller, Shad...","Amazing, Love, Love, Love it !!! All 5 STARS a...",5,Amazing Love Love Love it All 5 STARS all the...,"{'amazing': 1, 'love': 4, 'it': 7, 'all': 3, '...",1.0
97325,Freemie Hands-Free Concealable Breast Pump Col...,I absolutely love this product. I work as a C...,5,I absolutely love this product I work as a Cu...,"{'i': 38, 'absolutely': 1, 'love': 1, 'this': ...",1.0
119182,Roan Rocco Classic Pram Stroller 2-in-1 with B...,Great Pram Rocco!!!!!!I bought this pram from ...,5,Great Pram RoccoI bought this pram from Europe...,"{'great': 4, 'pram': 10, 'roccoi': 1, 'bought'...",1.0
133651,"Britax 2012 B-Agile Stroller, Red",[I got this stroller for my daughter prior to ...,4,I got this stroller for my daughter prior to t...,"{'i': 22, 'got': 3, 'this': 17, 'stroller': 22...",1.0
80155,"Simple Wishes Hands-Free Breastpump Bra, Pink,...","I just tried this hands free breastpump bra, a...",5,I just tried this hands free breastpump bra an...,"{'i': 23, 'just': 4, 'tried': 2, 'this': 4, 'h...",1.0
50315,"P'Kolino Silly Soft Seating in Tias, Green",I've purchased both the P'Kolino Little Reader...,4,Ive purchased both the PKolino Little Reader C...,"{'ive': 1, 'purchased': 1, 'both': 3, 'the': 3...",1.0
168697,Graco FastAction Fold Jogger Click Connect Str...,Graco's FastAction Jogging Stroller definitely...,5,Gracos FastAction Jogging Stroller definitely ...,"{'gracos': 1, 'fastaction': 2, 'jogging': 2, '...",1.0
52631,Evenflo X Sport Plus Convenience Stroller - Ch...,After seeing this in Parent's Magazine and rea...,5,After seeing this in Parents Magazine and read...,"{'after': 2, 'seeing': 1, 'this': 12, 'in': 6,...",1.0
114796,"Fisher-Price Cradle 'N Swing, My Little Snuga...",My husband and I cannot state enough how much ...,5,My husband and I cannot state enough how much ...,"{'my': 1, 'husband': 1, 'and': 19, 'i': 3, 'ca...",1.0
168081,Buttons Cloth Diaper Cover - One Size - 8 Colo...,"We are big Best Bottoms fans here, but I wante...",4,We are big Best Bottoms fans here but I wanted...,"{'we': 3, 'are': 6, 'big': 2, 'best': 13, 'bot...",1.0


In [37]:
X_test.tail(10)

,name,review,rating,review_clean,word_count,prob
10677,Philips AVENT Newborn Starter Set,"It's 3am in the morning and needless to say, t...",1,Its 3am in the morning and needless to say thi...,"{'its': 1, '3am': 3, 'in': 2, 'the': 11, 'morn...",1.150138e-10
113995,Motorola Digital Video Baby Monitor with Room ...,DO NOT BUY THIS BABY MONITOR!I purchased this ...,1,DO NOT BUY THIS BABY MONITORI purchased this m...,"{'do': 1, 'not': 2, 'buy': 1, 'this': 6, 'baby...",1.011637e-10
81332,Cloth Diaper Sprayer--styles may vary,I bought this sprayer out of desperation durin...,1,I bought this sprayer out of desperation durin...,"{'i': 12, 'bought': 1, 'this': 3, 'sprayer': 2...",4.088883e-11
53207,Safety 1st High-Def Digital Monitor,We bought this baby monitor to replace a diffe...,1,We bought this baby monitor to replace a diffe...,"{'we': 1, 'bought': 1, 'this': 5, 'baby': 8, '...",3.460592e-11
94560,The First Years True Choice P400 Premium Digit...,Note: we never installed batteries in these un...,1,Note we never installed batteries in these uni...,"{'note': 1, 'we': 3, 'never': 1, 'installed': ...",4.717844e-13
155287,VTech Communications Safe &amp; Sounds Full Co...,"This is my second video monitoring system, the...",1,This is my second video monitoring system the ...,"{'this': 12, 'is': 15, 'my': 6, 'second': 3, '...",2.319766e-13
48694,Adiri BPA Free Natural Nurser Ultimate Bottle ...,I will try to write an objective review of the...,2,I will try to write an objective review of the...,"{'i': 26, 'will': 4, 'try': 2, 'to': 29, 'writ...",1.639712e-13
77072,Safety 1st Exchangeable Tip 3 in 1 Thermometer,I thought it sounded great to have different t...,1,I thought it sounded great to have different t...,"{'i': 18, 'thought': 1, 'it': 8, 'sounded': 1,...",8.425898e-14
120209,Levana Safe N'See Digital Video Baby Monitor w...,This is the first review I have ever written o...,1,This is the first review I have ever written o...,"{'this': 9, 'is': 7, 'the': 21, 'first': 3, 'r...",2.324526e-15
16042,Fisher-Price Ocean Wonders Aquarium Bouncer,We have not had ANY luck with Fisher-Price pro...,2,We have not had ANY luck with FisherPrice prod...,"{'we': 3, 'have': 3, 'not': 5, 'had': 1, 'any'...",1.038830e-15


In [38]:
X_test.loc[83234]['review']

'My Experience: Babykicks Inserts failure vs RAVING Success of Thirsties insertsPurchased the Thirsties Hemp inserts 2-Pack, Lg 18-40lbs and the BabyKicks 3 Pack Joey Bunz Hemp Inserts, as both products seemed equally matched based on reviews. Put each brand to the "overnight" diapering test over a week, alternating nights. (CD\'ing fleece liner over 4x8x4 unbleached indian cotton premium prefolds with hemp inserts wrapped outside, then a Thirsties Duo Wrap cover size 2 over.) Both brands of inserts were prepped in same wash loads with same detergent, same dryers all at same time prior to the test.THE RESULTS?  Thirsties won and Babykicks was NOT remotely CLOSE!The Babykicks Hemp Inserts, when compared to the Thirsties inserts, performed little better than paper towel! The little one was completely SOAKED after a mere 3-4 hours on nights the Babykicks. And not just the baby, but the mattress, diaper cover exterior, EVERYTHING. (**On a side note, I must commend the Thristies Duo Diaper 

## Compute accuracy of the classifier

We will now evaluate the accuracy of the trained classifer. Recall that the accuracy is given by


$$
\mbox{accuracy} = \frac{\mbox{# correctly classified examples}}{\mbox{# total examples}}
$$

This can be computed as follows:

* **Step 1:** Use the trained model to compute class predictions (**Hint:** Use the `predict` method)
* **Step 2:** Count the number of data points when the predicted class labels match the ground truth labels (stored on X_testy).
* **Step 3:** Divide the total number of correct predictions by the total number of data points in the dataset.

Complete the function below to compute the classification accuracy:

In [39]:
scores = sentiment_model.predict(test_matrix)
scores = pd.DataFrame(scores)
y_hat = scores[0]
y_hat.sample(n=10)

18218    1
31614    1
700      1
15525    1
18719    1
22303    1
7858     1
19180    1
575     -1
6173     1
Name: 0, dtype: int64

In [40]:
X_test['Prediction'] = y_hat.values

In [41]:
len(X_test[X_test['Prediction'] == X_testy.values]) / len(X_test)

0.9323854091672666

# How does the model looks like?

In [42]:
sentiment_model.coef_

array([[-1.23647698e+00,  2.05856855e-04,  2.54473692e-02, ...,
         1.16368121e-02,  3.11529309e-03, -7.03629882e-05]])

Check the weight of the words on the model

In [43]:
Words = pd.DataFrame.from_dict(vectorizer.vocabulary_,'index')
Words['coef'] = sentiment_model.coef_[0]
# help(pd.DataFrame.from_dict)

In [44]:
Words = Words.reset_index()
Words.rename(columns={'index':'Word',0:'Count','coef':'Coef'},inplace=True)
Words.head()

,Word,Count,Coef
0,it,57486,-1.236477
1,came,21721,0.000206
2,early,37328,0.025447
3,and,10505,0.006200
4,was,116799,0.000046


In [55]:
Words.sort_values(by = 'Coef', ascending = False, inplace = True)
Words.head(15)

,Word,Count,Coef
10112,seatcould,92746,1.993880
80935,70th,6592,1.985279
40372,seemy,93302,1.948417
36155,arg,11830,1.923142
81178,picafter,79690,1.902525
91892,groing,49278,1.897935
78983,andstainless,10563,1.863449
89902,screeeeeeeam,92399,1.842767
76454,barebones,14997,1.842434
61811,crumns,30400,1.835036


In [54]:
Words.tail(15)

,Word,Count,Coef
72383,weekhighly,117695,-2.040320
89506,bornthey,18665,-2.075778
55481,fruitsweets,45888,-2.173985
34453,ladybugs,60251,-2.186564
113688,childconsnot,24200,-2.274611
81600,klack,59816,-2.281241
27609,bottlessippys,18945,-2.290332
81610,elecare,38340,-2.295021
114477,105installation,278,-2.322137
34484,receipient,87176,-2.429239
